In [0]:
import numpy as np
import os
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Transfer learning: fine-tuning an existing model

In this notebook, we look at a way to use a pre-trained model for a different task. Namely, we will take a vgg16 neural network (https://neurohive.io/en/popular-networks/vgg16/), remove its fully connected layers, add new connected layers with the right ouput format and retrain only these layers to a classification task on flowers.
We take vgg16 trained on Imagenet (a very large database of images of different categories), so that its filters are already trained to 'see' and extract meaningful features from the images. 

#### Because the computations which follow are intensive, please open this notebook on google colab. Google colab is a free jupter environement with free GPU acceleration (up to a certain time limit). **When opening a notebook in Google Collab, change the runtime type to GPU (default is CPU-only)**

## Loading the data

#### We provide below a method `load_flowers_data` which loads the data from a google drive shared folder https://drive.google.com/open?id=10vMxWm37w8ZioysTBOnA3FSj2RqWV2Qa. Please make sure to add this shared folder to your drive. 


When running the next cell, you will be prompted to go into your google account and to copy/paste a validation code allowing your drive to be available from this notebook in Google Collab environment. The loading of the images may take several minutes. Note that as a first step in processing the images, we resize them all to 256x256.

You might need to remount the drive sometimes.

In [0]:
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive/')
def load_flowers_data():
    data_path = '/content/drive/My Drive/06-Deep-Learning/02/data/flowers'
    classes = {'daisy':0, 'dandelion':1, 'rose':2}
    imgs = []
    labels = []
    for (cl, i) in classes.items():
        images_path = [elt for elt in os.listdir(os.path.join(data_path, cl)) if elt.find('.jpg')>0]
        for img in tqdm(images_path[:300]):
          path = os.path.join(data_path, cl, img)
          if os.path.exists(path):
            image = Image.open(path)
            image = image.resize((256, 256))
            imgs.append(np.array(image))
            labels.append(i)

    x = np.array(imgs)
    num_classes = len(set(labels))
    y = to_categorical(labels, num_classes)

    # Finally we shuffle:
    p = np.random.permutation(len(x))
    x, y = x[p], y[p]

    first_split = int(len(imgs) / 6.)
    second_split = first_split + int(len(imgs) * 0.2)
    x_test, x_val, x_train = x[:first_split], x[first_split:second_split], x[second_split:]
    y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]

    return x_train, y_train, x_val, y_val, x_test, y_test, num_classes

Use the `load_flowers_data` method and load the train, test and val images and labels as well as the number of classes. Check the dimensions of the obtained arrays. (if you encounter problems loading the files and with google drive, ask for help)

#### Plot some images, print the corresponding classes.

#### Complete the methods below, which compile a model and build a model from vgg16.

This should include the following steps : 
- 1) find how to make the layers of vgg16 non trainable
- 2) declare two dense layers. The first should take as input the output of the vgg16 model (check its shape!) and the second should output a classification over 3 classes (so 3 neurons with softmax activation)
- 3) take the input tensor, and apply vgg16 and your two layers to it. This gives you an out tensor.
- 4) Then do model = Model(input, out) : this defines your model !

In [0]:
def compile_model(model):
    opt = Adam(learning_rate=1e-4)
    model.compile(loss=?????,
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

def build_model():
    vgg16_no_top = VGG16(weights="imagenet", include_top=False, input_shape=x_train[0].shape)

    # Set vgg16 layers to be non trainable

    # Input of your model
    inp = Input(shape=x_train[0].shape)
    # Complete !
    ...
    out =  ?

    model = Model(inp, out)
    model = compile_model(model)
    return model


In [0]:
model = build_model()
model.summary()

The VGG16 model was trained on images which were preprocessed in a specific way. Apply this processing to the images here using the method `preprocess_input` that you can import from `tensorflow.keras.applications.vgg16`.

#### Now estimate the model, with an early stopping criterion on the validation accuracy. (provide the validation data)

Plot the history of this model training.


Evaluate the model accuracy on the test set. What is the chance level on this classification task (i.e. accuracy of a random classifier).

Find and plot some examples which were not correctly classified.

#### Perform the same tasks as before but this time using data augmentation. Do you see an improvement ?

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(????) # choose your augmentation


# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)

model_data_aug = build_model()

train_flow = datagen.flow(x_train, y_train, batch_size=16)
val_flow = datagen.flow(x_val, y_val, batch_size=16)

es = EarlyStopping(???)

history_data_aug = model_data_aug.fit_generator(????)

#### Suggest at least 4 ideas to improve the model test accuracy:
